In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from jupyterthemes import jtplot

jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False) 

In [5]:
twitter_df = pd.read_csv("twitter.csv")
twitter_df.drop(['id'],axis=1,inplace=True)   # removing id column since id,index gives the same data
print(twitter_df)

       label                                              tweet
0          0   @user when a father is dysfunctional and is s...
1          0  @user @user thanks for #lyft credit i can't us...
2          0                                bihday your majesty
3          0  #model   i love u take with u all the time in ...
4          0             factsguide: society now    #motivation
...      ...                                                ...
31957      0  ate @user isz that youuu?ðððððð...
31958      0    to see nina turner on the airwaves trying to...
31959      0  listening to sad songs on a monday morning otw...
31960      1  @user #sikh #temple vandalised in in #calgary,...
31961      0                   thank you @user for you follow  

[31962 rows x 2 columns]


In [11]:
twitter_df_1s = twitter_df.loc[twitter_df['label'] == 1]
print(twitter_df_1s.count())

twitter_df_0s = twitter_df.loc[twitter_df['label'] == 0]
print(twitter_df_0s.count())

label    2242
tweet    2242
dtype: int64
label    29720
tweet    29720
dtype: int64


In [12]:
twitter_df['length'] = twitter_df['tweet'].apply(len)
twitter_df

,label,tweet,length
0,0,@user when a father is dysfunctional and is s...,102
1,0,@user @user thanks for #lyft credit i can't us...,122
2,0,bihday your majesty,21
3,0,#model i love u take with u all the time in ...,86
4,0,factsguide: society now #motivation,39
...,...,...,...
31957,0,ate @user isz that youuu?ðððððð...,68
31958,0,to see nina turner on the airwaves trying to...,131
31959,0,listening to sad songs on a monday morning otw...,63
31960,1,"@user #sikh #temple vandalised in in #calgary,...",67
